In [ ]:
pip install PyPDF2 pandas tqdm google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.1 MB/s eta 0:00:00


In [ ]:
import os
import re
import csv
import time
import random
import logging
import pandas as pd
from typing import List, Dict, Tuple, Any
from tqdm.notebook import tqdm  # Colab için notebook tqdm versiyonu
from PyPDF2 import PdfReader
import google.generativeai as genai
from google.api_core.exceptions import ResourceExhausted, InvalidArgument
from google.colab import files  # Colab'a dosya yüklemek için
from pathlib import Path
from getpass import getpass  # API anahtarını güvenli şekilde almak için

In [ ]:
# Loglama yapılandırması
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("anayasa_qa_generator.log"),
        logging.StreamHandler()
    ]
)

In [ ]:
logger = logging.getLogger(__name__)

In [ ]:
class AnayasaQAGenerator:
    """Türkiye Cumhuriyeti Anayasası'ndan soru-cevap çiftleri üreten sınıf."""

    def __init__(self, pdf_path: str, api_key: str, page_limit: int = 72):
        """
        Args:
            pdf_path (str): Anayasa PDF dosyasının yolu
            api_key (str): Google AI API anahtarı
            page_limit (int): İşlenecek maksimum sayfa sayısı
        """
        self.pdf_path = pdf_path
        self.page_limit = page_limit
        self.qa_pairs = []

        # Google Generative AI API'sini yapılandır
        genai.configure(api_key=api_key)

        # Gemini-1.5-Flash modelini yapılandır
        self.model = genai.GenerativeModel('gemini-1.5-flash')

        logger.info(f"AnayasaQAGenerator initialized with PDF: {pdf_path}")

    def extract_text_from_pdf(self) -> List[Dict[str, Any]]:
        """PDF'ten metni çıkarır ve sayfa başına bir sözlük olarak döndürür.

        Returns:
            List[Dict[str, Any]]: Her sayfa için metin ve sayfa numarası içeren sözlük listesi
        """
        pages = []

        try:
            pdf = PdfReader(self.pdf_path)
            total_pages = min(len(pdf.pages), self.page_limit)

            logger.info(f"Extracting text from {total_pages} pages")

            for i in tqdm(range(total_pages), desc="PDF'ten metin çıkarılıyor"):
                text = pdf.pages[i].extract_text()
                # Gereksiz boşlukları temizle
                text = re.sub(r'\s+', ' ', text).strip()

                # Sayfa çok kısaysa (örneğin sadece başlık varsa) atla
                if len(text) < 100:
                    logger.warning(f"Page {i+1} has very little text ({len(text)} chars), might be skipped")

                pages.append({
                    "page_number": i + 1,
                    "text": text
                })

            logger.info(f"Successfully extracted text from {len(pages)} pages")
            return pages

        except Exception as e:
            logger.error(f"Error extracting text from PDF: {str(e)}")
            raise

    def generate_qa_prompt(self, text: str, page_number: int) -> str:
        """Verilen metinden soru-cevap çiftleri oluşturmak için bir prompt oluşturur.

        Args:
            text (str): Sayfa metni
            page_number (int): Sayfa numarası

        Returns:
            str: Oluşturulmuş prompt
        """
        prompt = f"""Sen Türkiye Cumhuriyeti Anayasası konusunda uzman bir hukuk profesörüsün.
Aşağıda Türkiye Cumhuriyeti Anayasası'ndan bir bölüm verilmiştir (Sayfa {page_number}).
Bu metinden elde edilebilecek, anlamlı ve doğru soru-cevap çiftleri oluştur.

Her soru-cevap çifti şu formatta olmalıdır:
SORU: [Anayasa metnine dayalı bir soru]
CEVAP: [Sorunun kapsamlı cevabı, sadece verilen metne dayalı]

Önemli Kurallar:
1. Sorular ve cevaplar SADECE verilen metne dayanmalıdır.
2. Gerçeklere aykırı veya metinde olmayan bilgiler içeren soru-cevap çiftleri oluşturma.
3. Cevaplar tam, açık ve anlaşılır olmalıdır.
4. Verilen metinden maksimum sayıda anlamlı soru-cevap çifti oluştur.
5. Sorular, anayasa maddelerinin anlaşılmasına yardımcı olacak bilgi aramaya yönelik olmalıdır.
6. Cevaplar, verilen metni aşmamalı ancak metni tam olarak kapsayacak şekilde detaylı olmalıdır.

İŞTE METİN:
{text}

Şimdi, bu metinden maksimum sayıda, anlamlı ve doğru soru-cevap çiftlerini üret. Sadece SORU: ve CEVAP: formatını kullan.
"""
        return prompt

    def extract_qa_pairs(self, response_text: str) -> List[Dict[str, str]]:
        """Model yanıtından soru-cevap çiftlerini çıkarır.

        Args:
            response_text (str): Model yanıtı

        Returns:
            List[Dict[str, str]]: Soru-cevap çiftleri listesi
        """
        qa_pairs = []

        # Soru-cevap çiftlerini bul
        pattern = r"SORU:\s*(.*?)\s*CEVAP:\s*(.*?)(?=SORU:|$)"
        matches = re.finditer(pattern, response_text, re.DOTALL)

        for match in matches:
            question = match.group(1).strip()
            answer = match.group(2).strip()

            # Boş soru veya cevapları atla
            if not question or not answer:
                continue

            qa_pairs.append({
                "question": question,
                "answer": answer
            })

        return qa_pairs

    def process_page(self, page: Dict[str, Any], retry_count: int = 3) -> List[Dict[str, str]]:
        """Tek bir PDF sayfasını işler ve soru-cevap çiftleri oluşturur.

        Args:
            page (Dict[str, Any]): Sayfa içeriği ve numarası
            retry_count (int): Hata durumunda yeniden deneme sayısı

        Returns:
            List[Dict[str, str]]: Sayfadan üretilen soru-cevap çiftleri
        """
        page_text = page["text"]
        page_number = page["page_number"]

        if len(page_text) < 100:  # Çok kısa sayfaları atla
            logger.info(f"Skipping page {page_number} as it has insufficient text")
            return []

        prompt = self.generate_qa_prompt(page_text, page_number)

        for attempt in range(retry_count):
            try:
                logger.info(f"Processing page {page_number}, attempt {attempt+1}/{retry_count}")

                # Gemini API'sini çağır
                response = self.model.generate_content(prompt)

                if not response.text:
                    logger.warning(f"Empty response for page {page_number}, attempt {attempt+1}")
                    time.sleep(2)  # API hız sınırı aşımını önlemek için bekleme
                    continue

                # Yanıttan soru-cevap çiftlerini çıkar
                qa_pairs = self.extract_qa_pairs(response.text)

                # Her çifte sayfa numarası ekle
                for qa in qa_pairs:
                    qa["page_number"] = page_number

                logger.info(f"Successfully generated {len(qa_pairs)} QA pairs from page {page_number}")
                return qa_pairs

            except ResourceExhausted:
                logger.warning(f"API quota exceeded for page {page_number}, waiting before retry")
                time.sleep(60)  # Kotayı aştıysak bir dakika bekle

            except InvalidArgument as e:
                # Prompt çok uzunsa kısalt
                if "exceeds maximum" in str(e):
                    logger.warning(f"Prompt too long for page {page_number}, truncating")
                    # Metni yarıya düşür ve tekrar dene
                    text_length = len(page_text)
                    page_text = page_text[:text_length//2]
                    prompt = self.generate_qa_prompt(page_text, page_number)
                else:
                    logger.error(f"Invalid argument error for page {page_number}: {str(e)}")
                    time.sleep(2)

            except Exception as e:
                logger.error(f"Error processing page {page_number}, attempt {attempt+1}: {str(e)}")
                time.sleep(5)  # Hata durumunda 5 saniye bekle

        logger.error(f"Failed to process page {page_number} after {retry_count} attempts")
        return []

    def process_all_pages(self) -> List[Dict[str, str]]:
        """Tüm PDF sayfalarını işler ve soru-cevap çiftleri oluşturur.

        Returns:
            List[Dict[str, str]]: Tüm sayfalardan üretilen soru-cevap çiftleri
        """
        all_qa_pairs = []

        # PDF'ten metni çıkar
        pages = self.extract_text_from_pdf()

        # Her sayfayı işle
        for page in tqdm(pages, desc="Sayfalar işleniyor"):
            # Her sayfada rastgele bir gecikme ekle (API hız sınırlamasını önlemek için)
            time.sleep(random.uniform(1, 3))

            page_qa_pairs = self.process_page(page)
            all_qa_pairs.extend(page_qa_pairs)

            # Her 10 sayfada bir ilerleme bilgisi yaz
            if page["page_number"] % 10 == 0:
                logger.info(f"Progress: {page['page_number']}/{len(pages)} pages processed, {len(all_qa_pairs)} QA pairs generated so far")

        logger.info(f"Total QA pairs generated: {len(all_qa_pairs)}")
        self.qa_pairs = all_qa_pairs
        return all_qa_pairs

    def save_to_csv(self, output_path: str) -> None:
        """Soru-cevap çiftlerini CSV dosyasına kaydeder.

        Args:
            output_path (str): Çıktı CSV dosyasının yolu
        """
        if not self.qa_pairs:
            logger.warning("No QA pairs to save")
            return

        try:
            # DataFrame oluştur
            df = pd.DataFrame(self.qa_pairs)

            # CSV'ye kaydet
            df.to_csv(output_path, index=False, encoding='utf-8')

            logger.info(f"Successfully saved {len(self.qa_pairs)} QA pairs to {output_path}")

            # Colab için CSV'yi indir
            try:
                files.download(output_path)
                print(f"CSV dosyası indirilmeye hazır: {output_path}")
            except:
                print(f"CSV dosyası oluşturuldu ama indirilemedi. Dosya konumu: {output_path}")

        except Exception as e:
            logger.error(f"Error saving QA pairs to CSV: {str(e)}")
            raise

In [ ]:
# Ana uygulama kodu - Doğrudan Colab hücrelerinde çalıştırılacak şekilde tasarlandı
def run_anayasa_qa_generator():
    print("Türkiye Cumhuriyeti Anayasası'ndan Soru-Cevap Çiftleri Üretme Uygulaması")
    print("-" * 70)
    print("1. Adım: PDF dosyasını yükleyin")

    # PDF dosyasını yükle
    uploaded = files.upload()

    if not uploaded:
        print("Dosya yüklenmedi. Lütfen tekrar deneyin.")
        return

    pdf_filename = list(uploaded.keys())[0]
    print(f"Yüklenen dosya: {pdf_filename}")

    # API anahtarını güvenli bir şekilde al
    print("\n2. Adım: Google AI API anahtarını girin (yazarken görünmeyecektir)")
    api_key = getpass("Google AI API anahtarı: ")

    if not api_key:
        print("API anahtarı girilmedi. İşlem iptal ediliyor.")
        return

    # Sayfa limitini al
    page_limit_str = input("\n3. Adım: İşlenecek maksimum sayfa sayısını girin (varsayılan: 72): ")
    page_limit = 72
    if page_limit_str.strip() and page_limit_str.isdigit():
        page_limit = int(page_limit_str)

    # Çıktı dosya adını al
    output_file = input("\n4. Adım: Çıktı CSV dosyasının adını girin (varsayılan: anayasa_qa_pairs.csv): ")
    if not output_file.strip():
        output_file = "anayasa_qa_pairs.csv"
    elif not output_file.endswith(".csv"):
        output_file += ".csv"

    print("\n5. Adım: İşlem başlatılıyor...")
    print(f"  - PDF dosyası: {pdf_filename}")
    print(f"  - İşlenecek sayfa sayısı: {page_limit}")
    print(f"  - Çıktı dosyası: {output_file}")
    print("-" * 70)

    try:
        # QA Generator'ı başlat
        generator = AnayasaQAGenerator(
            pdf_path=pdf_filename,
            api_key=api_key,
            page_limit=page_limit
        )

        # Tüm sayfaları işle
        generator.process_all_pages()

        # Sonuçları CSV'ye kaydet
        generator.save_to_csv(output_file)

        print(f"\nİşlem tamamlandı! {len(generator.qa_pairs)} soru-cevap çifti oluşturuldu.")
        print(f"Sonuçlar {output_file} dosyasına kaydedildi.")

    except Exception as e:
        print(f"\nHata oluştu: {str(e)}")
        import traceback
        traceback.print_exc()


In [ ]:
# Bu kodu Colab'da çalıştırın
run_anayasa_qa_generator()

Türkiye Cumhuriyeti Anayasası'ndan Soru-Cevap Çiftleri Üretme Uygulaması
----------------------------------------------------------------------
1. Adım: PDF dosyasını yükleyin


Saving Anayasa.pdf to Anayasa.pdf
Yüklenen dosya: Anayasa.pdf

2. Adım: Google AI API anahtarını girin (yazarken görünmeyecektir)
Google AI API anahtarı: ··········

3. Adım: İşlenecek maksimum sayfa sayısını girin (varsayılan: 72): 72

4. Adım: Çıktı CSV dosyasının adını girin (varsayılan: anayasa_qa_pairs.csv): anayasa_qa_pairs.csv

5. Adım: İşlem başlatılıyor...
  - PDF dosyası: Anayasa.pdf
  - İşlenecek sayfa sayısı: 72
  - Çıktı dosyası: anayasa_qa_pairs.csv
----------------------------------------------------------------------


PDF'ten metin çıkarılıyor:   0%|          | 0/72 [00:00<?, ?it/s]

Sayfalar işleniyor:   0%|          | 0/72 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

CSV dosyası indirilmeye hazır: anayasa_qa_pairs.csv

İşlem tamamlandı! 875 soru-cevap çifti oluşturuldu.
Sonuçlar anayasa_qa_pairs.csv dosyasına kaydedildi.


In [ ]:
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', None)  # görüntüleme ayarlarını yapıyoruz
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
df=pd.read_csv('anayasa_qa_pairs.csv')

In [ ]:
df.head()

,question,answer,page_number
0,Anayasanın kabul tarihi nedir ve hangi sayılı ...,Anayasa 18/10/1982 tarihinde kabul edilmiş ve ...,1
1,Anayasa hangi amaçla belirlenmiştir?,Türk Vatanı ve Milletinin ebedi varlığını ve Y...,1
2,Anayasanın hazırlanmasında hangi anlayış ve il...,Türkiye Cumhuriyetinin kurucusu Atatürk'ün bel...,1
3,Anayasanın Türkiye Cumhuriyeti için belirlediğ...,"Türkiye Cumhuriyetinin ebedi varlığı, refahı, ...",1
4,Anayasaya göre egemenlik kime aittir?,Egemenlik kayıtsız şartsız Türk Milletine aittir.,1


In [ ]:
df.to_csv('anayasa_qa_pairs_clean.csv', index=False, encoding='utf-8-sig')